In [1]:
# Import required libraries for data manipulation and analysis
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr

In [2]:
#Import required sklearn functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.inspection import permutation_importance
from collections import defaultdict

In [3]:
#Import sklearn classifiers
from sklearn.ensemble import RandomForestClassifier

In [4]:
#Import library to oversample 
from imblearn.over_sampling import RandomOverSampler

In [5]:
#Import RDKit and Mordred libraries
from rdkit import Chem
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

In [6]:
# Sets Pandas Display to Monitor Code
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [7]:
# Create Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True)

In [8]:
#Functions used in the study

#Remove those numbers from analysis data
def filter_rows_by_values1(df, col, values):
    return df[~df[col].isin(values)]

#Remove those numbers from analysis data
def filter_rows_by_values2(df, col, values):
    return df[df[col].isin(values)]

#Get Mordred calcs
def get_Mordred(data_input):
    # Assigns Reactants Mordred Info
    reactants = data_input['Substrate']
    
    reactants_mol_list = []
    for inChi_reactants in reactants:
      reactants_mol = Chem.MolFromInchi(inChi_reactants)
      reactants_mol_list.append(reactants_mol)

    # Puts reactants into Pandas Type
    reactant_data = []
    reactant_data = calc.pandas(reactants_mol_list)
       
    #Joins Mordred parameters with experimental, atomic charges, and JChem for Excel parameters
    add_reactants = pd.concat((data_input, reactant_data), axis=1)
    
    #Force any non-numeric entries as NaN and replace them with 0
    int_data = add_reactants.apply(pd.to_numeric, errors='coerce')
    
    output = int_data.fillna(0)#, inplace=True)

    return output

#Remove zero varience
def remove_zero_varience(values):
   sel = VarianceThreshold()
   _ = sel.fit(values)
   mask = sel.get_support()
   values = values.loc[:,mask] 
   return values

def remove_95correlated(correlated):
    #Remove any features that are greater than 95% correlated
    corr_matrix = correlated.corr()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

    correlated = correlated.drop(to_drop, axis = 1)
    corr_matrix = correlated.corr()
    return correlated

def remove_nonimportant(X_values, y_values):
    # Specifys Random Forest and the Number of Trees, SelectFromModel will
    # select features which are most important
    feature_names = [f"feature {i}" for i in range(X_values.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(X_values, y_values)

    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time

    threshold = np.sort(importances)[-100]
    
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(X_values, y_values)

    # Select the final features set 
    sel.get_support()
    selected_feat= X_values.columns[(sel.get_support())]

    # Prints the names of the final selected features
    print(selected_feat)
    X_values = X_values[selected_feat]
    
    return X_values

def dendrogram(X_values, y):
    corr = spearmanr(X_values).correlation
    # Ensure the correlation matrix is symmetric
    corr = (corr + corr.T) / 2
    np.fill_diagonal(corr, 1)
    distance_matrix = 1 - np.abs(corr)
    dist_linkage = hierarchy.ward(squareform(distance_matrix))
  
    trained_cluster_ids = hierarchy.fcluster(dist_linkage, y, criterion="distance")
    trained_cluster_id_to_feature_ids = defaultdict(list) 
    for idx, trained_cluster_id in enumerate(trained_cluster_ids):
        trained_cluster_id_to_feature_ids[trained_cluster_id].append(idx)
    
    trained_selected_features = [v[0] for v in trained_cluster_id_to_feature_ids.values()]
    final_selected_features = X_values.columns[trained_selected_features]
    X_train = X_values[final_selected_features]
    return X_train

def classificationMetrics(results, y_test, pred):
    acc = accuracy_score(y_test, pred)
    prec = precision_score(y_test, pred, average=None, zero_division=0)
    recall = recall_score(y_test, pred, average=None)
    F1 = f1_score(y_test, pred, average=None)           
    #Calculate confusion matrix
    cf_matrix = confusion_matrix(y_test, pred)
    cf_matrix = np.reshape(cf_matrix,(1,4))
    comb = np.concatenate((x, y, cf_matrix, acc, prec, recall, F1), axis=None)
    comb = [comb]
    results = results.append(pd.DataFrame(comb, columns=results.columns), ignore_index=True)
    return results


In [9]:
# Read Training/Test data input File
data = pd.read_csv('BorylationTrainingTest 9-12-24.csv')

#group the compounds by numbers
data['grouped'] = data.groupby('Substrate', sort=False).ngroup()
grouped_sub = data[['grouped','Substrate','Product_Ratio']]
#Convert substrates to Mordred features
mordred_data = get_Mordred(grouped_sub)

mordred_data.head()

100%|██████████| 1033/1033 [01:32<00:00, 11.20it/s]


,grouped,Substrate,Product_Ratio,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,VE3_A,VR1_A,VR2_A,VR3_A,nAromAtom,nAromBond,nAtom,nHeavyAtom,nSpiro,nBridgehead,...,MWC05,MWC06,MWC07,MWC08,MWC09,MWC10,TMWC10,SRW02,SRW03,SRW04,SRW05,SRW06,SRW07,SRW08,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0,0.0,1,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,1.189693,2.876615,2.673468,0.334183,0.760233,21.482988,2.685374,2.844118,0,0,26,8,0,0,...,5.129899,5.758902,6.385194,7.016610,7.645398,8.277158,66.911677,2.708050,0.0,3.663562,0.0,4.762174,0.0,5.926926,0.0,7.126891,32.187603,114.140851,4.390033,84,5,26.0,24.0,3.500000,2.25
1,0,0.0,0,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,1.189693,2.876615,2.673468,0.334183,0.760233,21.482988,2.685374,2.844118,0,0,26,8,0,0,...,5.129899,5.758902,6.385194,7.016610,7.645398,8.277158,66.911677,2.708050,0.0,3.663562,0.0,4.762174,0.0,5.926926,0.0,7.126891,32.187603,114.140851,4.390033,84,5,26.0,24.0,3.500000,2.25
2,0,0.0,0,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,1.189693,2.876615,2.673468,0.334183,0.760233,21.482988,2.685374,2.844118,0,0,26,8,0,0,...,5.129899,5.758902,6.385194,7.016610,7.645398,8.277158,66.911677,2.708050,0.0,3.663562,0.0,4.762174,0.0,5.926926,0.0,7.126891,32.187603,114.140851,4.390033,84,5,26.0,24.0,3.500000,2.25
3,0,0.0,0,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,1.189693,2.876615,2.673468,0.334183,0.760233,21.482988,2.685374,2.844118,0,0,26,8,0,0,...,5.129899,5.758902,6.385194,7.016610,7.645398,8.277158,66.911677,2.708050,0.0,3.663562,0.0,4.762174,0.0,5.926926,0.0,7.126891,32.187603,114.140851,4.390033,84,5,26.0,24.0,3.500000,2.25
4,1,0.0,1,8.485281,8.468504,0,1,15.664838,2.101003,4.202006,15.664838,1.204988,3.380141,3.123485,0.240268,1.401314,59.248239,4.557557,4.344100,0,0,40,13,0,0,...,6.013715,6.748760,7.482682,8.223359,8.962007,9.704610,85.894968,3.218876,0.0,4.290459,0.0,5.533389,0.0,6.867974,0.0,8.259717,41.170416,185.214350,4.630359,300,12,48.0,48.0,5.361111,3.50


In [10]:
Results_df = pd.DataFrame(columns =  ['x', 'y',  "True Neg","False Pos","False Neg","True Pos",'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

maxacc_comb = pd.DataFrame()
val_tot = pd.DataFrame()
prod = pd.DataFrame()
test_index_total = pd.DataFrame()

model_columns = pd.DataFrame()
for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 200,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(mordred_data, "grouped", set_numbers)
    test_data = filter_rows_by_values2(mordred_data, "grouped", set_numbers)
  
    #Remove features that dont change
    training_data = remove_zero_varience(training_data)
    
    #Remove features that are more than 95% correlated
    training_data = remove_95correlated(training_data)
    
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio', axis = 1)
    training_X = training_X.drop('grouped', axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio', axis = 1)
    test_X = test_X.drop('grouped', axis = 1)
    test_y = test_data['Product_Ratio']
 
    #Remove features that are considered less important
    feature_names = [f"feature {i}" for i in range(training_X.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(training_X, training_y)
    
    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time
    
    threshold = np.sort(importances)[-100] 
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(training_X, training_y)
     
    # Select the reduced features set 
    sel.get_support()
    selected_feat= training_X.columns[(sel.get_support())]
    
    reduced1_X = training_X[selected_feat]
    test_X = test_X[selected_feat]
    
    #Apply over-sampling to dataset
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(reduced1_X, training_y) 
    
    for y in [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
              0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 
              0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35,
              0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 
              0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45,
              0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50,
              0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55]:
             
    
        #Make final training and test set and save them as df's  
        X_train = dendrogram(X_resampled, y)
        test_X = test_X[X_train.columns]
        training_columns_list = X_train.columns.tolist()
        training_columns_list = (x, y, training_columns_list)
        training_columns_list = (pd.DataFrame(training_columns_list).T)

        #Random Forest Classifier
        rfc = RandomForestClassifier(n_estimators=800,max_depth=9)
        rfc.fit(X_train, y_resampled)
        pred = rfc.predict(test_X)
        Results_df = classificationMetrics(Results_df, test_y, pred)

        #Evaluate model by going line by line
        ynew = rfc.predict(test_X)
        prediction_df = pd.DataFrame(ynew,  columns = [(x,y)])

        val_pred_T = prediction_df.T
        val_tot = val_tot.append(val_pred_T)

        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = Results_df.groupby('y')['acc'].mean()
        acc_std = Results_df.groupby('y')['acc'].std()
        precision_0_mean = Results_df.groupby('y')['precision 0'].mean()
        precision_0_std = Results_df.groupby('y')['precision 0'].std()
        precision_1_mean = Results_df.groupby('y')['precision 1'].mean()
        precision_1_std = Results_df.groupby('y')['precision 1'].std()
        recall_0_mean = Results_df.groupby('y')['recall 0'].mean()
        recall_0_std = Results_df.groupby('y')['recall 0'].std()
        recall_1_mean = Results_df.groupby('y')['recall 1'].mean()
        recall_1_std = Results_df.groupby('y')['recall 1'].std()
        F1_0_mean = Results_df.groupby('y')['F1 0'].mean()
        F1_0_std = Results_df.groupby('y')['F1 0'].std()
        F1_1_mean = Results_df.groupby('y')['F1 1'].mean()
        F1_1_std = Results_df.groupby('y')['F1 1'].std()
        true_neg_mean = Results_df.groupby('y')['True Neg'].mean()
        true_neg_std = Results_df.groupby('y')['True Neg'].std()
        false_pos_mean = Results_df.groupby('y')['False Pos'].mean()
        false_pos_std = Results_df.groupby('y')['False Pos'].std()        
        false_neg_mean = Results_df.groupby('y')['False Neg'].mean()
        false_neg_std = Results_df.groupby('y')['False Neg'].std()      
        true_pos_mean = Results_df.groupby('y')['True Pos'].mean() 
        true_pos_std = Results_df.groupby('y')['True Pos'].std()   
        

        average_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std,
                                   true_neg_mean, true_neg_std,
                                   false_pos_mean, false_pos_std,
                                   false_neg_mean, false_neg_std,
                                   true_pos_mean, true_pos_std], axis=1)
    
        average_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std', 'true_neg_mean', 'true_neg_std',
                                 'false_pos_mean', 'false_pos_std','false_neg_mean', 'false_neg_std',
                                 'true_pos_mean', 'true_pos_std']                                 

        maxacc = average_df[average_df.acc_mean == average_df.acc_mean.max()]
        maxacc_copy  = maxacc.copy()
        maxacc_copy['x_col'] = x
        
        model_columns = model_columns.append(training_columns_list)
        #print(x,y)
    test_index = pd.DataFrame(test_data.index.values)
    test_index_total = pd.concat([test_index_total, test_index],axis = 1)
    test_y = test_y.rename(x)
    prod = prod.append(test_y)
    maxacc_comb = maxacc_comb.append(maxacc_copy)  

total_results = val_tot.T
 
maxacc_comb.to_csv("Mordredonly.csv")
model_columns = model_columns.rename(columns = {0:'x', 1:'y', 2: 'features'})
model_columns = model_columns.drop_duplicates(subset = ['x','y'])
model_columns.to_csv("Mordredonly.csv", mode="a")

In [11]:
maxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,true_neg_mean,true_neg_std,false_pos_mean,false_pos_std,false_neg_mean,false_neg_std,true_pos_mean,true_pos_std,x_col
y,,,,,,,,,,,,,,,,,,,,,,,
0.35,0.600483,0.011844,0.831119,0.000594,0.232220,0.002363,0.633533,0.018894,0.462500,0.017678,0.718851,0.012179,0.309055,0.001965,105.800000,3.155243,61.200000,3.155243,21.500000,0.707107,18.500000,0.707107,1
0.25,0.608125,0.017526,0.833160,0.002238,0.259177,0.028706,0.635693,0.022514,0.498750,0.036702,0.720950,0.014641,0.340753,0.032111,100.050000,6.004165,57.450000,5.835238,20.050000,1.468081,19.950000,1.468081,2
0.25,0.647639,0.058745,0.839129,0.008795,0.266555,0.025679,0.693525,0.085465,0.447500,0.079965,0.757148,0.053517,0.330387,0.030041,116.600000,24.333318,50.066667,11.700084,22.100000,3.198599,17.900000,3.198599,3
0.25,0.649126,0.050761,0.839023,0.007600,0.263932,0.022669,0.697067,0.073998,0.441875,0.069657,0.759772,0.046405,0.327682,0.026363,117.350000,21.028125,49.900000,10.101536,22.325000,2.786299,17.675000,2.786299,4
0.25,0.646101,0.046106,0.836886,0.008123,0.262216,0.020667,0.694529,0.066997,0.440000,0.063286,0.757674,0.042041,0.326302,0.023922,115.780000,19.096201,50.020000,9.161788,22.400000,2.531435,17.600000,2.531435,5
0.25,0.625723,0.062589,0.836400,0.007529,0.259253,0.020008,0.661889,0.096217,0.470000,0.089466,0.735350,0.063800,0.330330,0.023635,109.200000,22.927151,54.466667,13.149037,21.200000,3.578656,18.800000,3.578656,6
0.25,0.613933,0.065320,0.839238,0.009946,0.255323,0.020939,0.642478,0.101935,0.491071,0.098635,0.723293,0.066765,0.331231,0.021996,106.600000,22.271968,58.400000,15.729968,20.357143,3.945409,19.642857,3.945409,7
0.55,0.620775,0.053565,0.840701,0.013480,0.254648,0.020472,0.652528,0.078426,0.485625,0.065817,0.732302,0.052721,0.331664,0.018383,109.537500,18.542421,57.712500,12.318756,20.575000,2.632682,19.425000,2.632682,8
0.55,0.624392,0.051505,0.841698,0.013013,0.252588,0.020157,0.658103,0.075571,0.478889,0.064902,0.736405,0.051024,0.328543,0.019462,111.811111,18.628174,57.411111,11.637690,20.844444,2.596069,19.155556,2.596069,9


In [12]:
final_selected_features = ['VR1_A', 'nAtom', 'ATS1Z', 'AATS1s', 'AATS2Z', 'AATS1i', 'AATS4i', 'AATS5i', 'AATSC6c', 'AATSC2d', 'AATSC4d', 'AATSC5s', 'AATSC3Z', 'AATSC5Z', 
                           'AATSC5v', 'AATSC5se', 'AATSC4p', 'MATS5c', 'MATS1d', 'MATS6se', 'MATS3p', 'GATS5d', 'GATS3se', 'BCUTdv-1l', 'RPCG', 
                           'AXp-0dv', 'AXp-6dv', 'AXp-7dv', 'IC3', 'SIC0', 'FilterItLogS', 'PEOE_VSA6', 'AMID_C']


#Loads validation dataset for borlation using the final reduced features 
unknownSubstrates=pd.read_csv('validation8-26-24.csv')

# Convert validation substrates Inchi's to Mordred and combine into Dataframe with atomic charges and JChem paramters
New_Substrate = unknownSubstrates['Substrate']
New_Substrate_mol_list = []
for inChi_New_Substrate in New_Substrate:
  New_Substrate_mol = Chem.MolFromInchi(inChi_New_Substrate)
  New_Substrate_mol_list.append(New_Substrate_mol)

New_Substrate_data = []
New_Substrate_data = calc.pandas(New_Substrate_mol_list)
New_Substrate_data = New_Substrate_data.apply(pd.to_numeric, errors='coerce')
New_Substrate_data.fillna(0, inplace=True)                                                                  
XnewSec = pd.concat((unknownSubstrates, New_Substrate_data), axis=1)
Xnew = XnewSec[final_selected_features]

val_tot = pd.DataFrame()

for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 200,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(mordred_data, "grouped", set_numbers)
    test_data = filter_rows_by_values2(mordred_data, "grouped", set_numbers)
   
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio' , axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio' , axis = 1)
    test_y = test_data['Product_Ratio']
   
    #Apply over-sampling to training set
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(training_X, training_y)    
    X_train = X_resampled[final_selected_features]

    #Random Forest Classifier
    rfc = RandomForestClassifier(n_estimators=800,max_depth=9)
    rfc.fit(X_train, y_resampled)
    
    #Evaluate the model on validation set
    ynew = rfc.predict(Xnew)
    validation_prediction_df = pd.DataFrame(ynew, columns = [(x)])
    validation_prediction_df.merge(validation_prediction_df, on=x)
    val_pred_T = validation_prediction_df.T
    val_tot = val_tot.append(val_pred_T)        

#Print the validation evaluations for model
unknownSubstrates_prod = unknownSubstrates['Product_Ratio']
total_val_results_transposed = val_tot.T
Val_results = pd.concat((unknownSubstrates_prod, total_val_results_transposed), axis=1)

#Write the results onto a CSV file 
#totalResults_df.to_csv("10Runs_FullResults.csv", index=False)
Val_results.to_csv("Mordredonly.csv", mode="a")
Val_results

100%|██████████| 81/81 [00:23<00:00,  3.46it/s]


,Product_Ratio,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
76,1,1,1,1,1,1,1,1,1,1,1
77,0,1,1,1,1,1,1,1,1,1,1
78,0,1,1,1,1,1,1,1,1,1,1
79,0,1,1,1,1,1,1,1,1,1,1
